In [2]:
import pandas as pd
from matplotlib.pyplot import yscale
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest         
from sklearn.feature_selection import chi2

data = pd.read_csv("out_ml_copy.csv")
#X = data.iloc[:,0:8]  #independent columns
#y = data.iloc[:,-4]    #target column i.e 

#bestfeatures = SelectKBest(score_func=chi2, k=10)
#fit = bestfeatures.fit(X,y)
#dfscores = pd.DataFrame(fit.scores_)
#dfcolumns = pd.DataFrame(X.columns)

PreProcessing 

In [55]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the CSV file into a pandas DataFrame
data = pd.read_csv('out_ml_original.csv')  # Update with your file name

# Convert the timestamp string column to datetime format
data['time:timestamp'] = pd.to_datetime(data['time:timestamp'])

# Extract relevant features from the timestamp
data['hour'] = data['time:timestamp'].dt.hour
data['minute'] = data['time:timestamp'].dt.minute
data['day_of_week'] = data['time:timestamp'].dt.dayofweek  # Monday=0, Sunday=6
# Add more datetime-related features as needed

# Drop the original timestamp column and any other columns that are not needed
data.drop(columns=['time:timestamp','TOC', 'case_id','name','DeviceType','Device'], inplace=True)

# Separate categorical and numerical columns
categorical_columns = ['activity']
numerical_columns = ['hour', 'minute', 'day_of_week']  # Add more numerical columns as needed

# Apply one-hot encoding to categorical columns
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_columns)],
    remainder='passthrough'
)

# Transform the data
X = preprocessor.fit_transform(data.drop(columns=['event_message']))
y = data['event_message']

# Print the first few rows of transformed data
print("Transformed Data:\n", X)

Transformed Data:
   (0, 17)	1.0
  (0, 22)	2.0
  (0, 23)	4.0
  (1, 3)	1.0
  (1, 22)	10.0
  (1, 23)	4.0
  (2, 1)	1.0
  (2, 22)	13.0
  (2, 23)	4.0
  (3, 3)	1.0
  (3, 22)	14.0
  (3, 23)	4.0
  (4, 20)	1.0
  (4, 22)	18.0
  (4, 23)	4.0
  (5, 4)	1.0
  (5, 22)	19.0
  (5, 23)	4.0
  (6, 14)	1.0
  (6, 22)	22.0
  (6, 23)	4.0
  (7, 14)	1.0
  (7, 22)	23.0
  (7, 23)	4.0
  (8, 1)	1.0
  :	:
  (1215, 9)	1.0
  (1215, 22)	99694.0
  (1215, 23)	4.0
  (1215, 24)	11.0
  (1215, 25)	36.0
  (1216, 14)	1.0
  (1216, 22)	99804.0
  (1216, 23)	4.0
  (1216, 24)	11.0
  (1216, 25)	36.0
  (1217, 14)	1.0
  (1217, 22)	99809.0
  (1217, 23)	4.0
  (1217, 24)	11.0
  (1217, 25)	36.0
  (1218, 14)	1.0
  (1218, 22)	99815.0
  (1218, 23)	4.0
  (1218, 24)	11.0
  (1218, 25)	36.0
  (1219, 4)	1.0
  (1219, 22)	99855.0
  (1219, 23)	4.0
  (1219, 24)	11.0
  (1219, 25)	37.0


In [56]:
k_best = 10  # Number of top features to select
bestfeatures = SelectKBest(score_func=chi2, k=k_best)
fit = bestfeatures.fit(X, y)

In [57]:
# Get scores and selected feature indices
# Get scores and selected feature indices
dfscores = pd.DataFrame(fit.scores_)
selected_indices = fit.get_support(indices=True)
selected_feature_names = preprocessor.transformers_[0][1].get_feature_names_out(input_features=categorical_columns)
# Print the scores and selected feature names
print("Scores of features:\n", dfscores)
print("Selected feature names:\n", selected_feature_names)

Scores of features:
                0
0   1.200000e+03
1   1.084000e+03
2   1.214000e+03
3   1.169000e+03
4   1.099000e+03
5   1.218000e+03
6   1.091000e+03
7   1.198000e+03
8   1.188000e+03
9   1.210000e+03
10  1.048000e+03
11  1.182000e+03
12  1.216000e+03
13  1.218000e+03
14  1.032000e+03
15  1.185000e+03
16  1.102000e+03
17  1.143000e+03
18  1.219000e+03
19  1.216000e+03
20  1.198000e+03
21  1.190000e+03
22  5.489580e+06
23  5.339287e-28
24  6.706887e+02
25  1.148426e+04
26           NaN
Selected feature names:
 ['activity_TOCBM' 'activity_TOCBR' 'activity_TOCBY' 'activity_TOCED'
 'activity_TOCFC' 'activity_TOCFCS' 'activity_TOCGR' 'activity_TOCHS'
 'activity_TOCLC' 'activity_TOCLF' 'activity_TOCLR' 'activity_TOCMN'
 'activity_TOCMS' 'activity_TOCMSS' 'activity_TOCNL' 'activity_TOCOG'
 'activity_TOCPA' 'activity_TOCPB' 'activity_TOCPBS' 'activity_TOCSE'
 'activity_TOCSR' 'activity_TOCWN']


In [58]:
#concat two dataframes for better visualization 
#featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#featureScores.columns = ['activity','DeviceType']  #naming the dataframe columns

In [59]:
dfscores = pd.DataFrame(fit.scores_)
selected_indices = fit.get_support(indices=True)
selected_feature_names = preprocessor.transformers_[0][1].get_feature_names_out(input_features=categorical_columns)

# Print the scores and selected feature names
print("Scores of features:\n", dfscores)
print("Selected feature names:\n", selected_feature_names)

# Concatenate feature scores and names
featureScores = pd.concat([pd.DataFrame(selected_feature_names, columns=['Feature']), dfscores], axis=1)

# Naming the dataframe columns
featureScores.columns = ['Feature', 'Score']

# Print the concatenated feature scores and names
print("Concatenated feature scores and names:\n", featureScores)


Scores of features:
                0
0   1.200000e+03
1   1.084000e+03
2   1.214000e+03
3   1.169000e+03
4   1.099000e+03
5   1.218000e+03
6   1.091000e+03
7   1.198000e+03
8   1.188000e+03
9   1.210000e+03
10  1.048000e+03
11  1.182000e+03
12  1.216000e+03
13  1.218000e+03
14  1.032000e+03
15  1.185000e+03
16  1.102000e+03
17  1.143000e+03
18  1.219000e+03
19  1.216000e+03
20  1.198000e+03
21  1.190000e+03
22  5.489580e+06
23  5.339287e-28
24  6.706887e+02
25  1.148426e+04
26           NaN
Selected feature names:
 ['activity_TOCBM' 'activity_TOCBR' 'activity_TOCBY' 'activity_TOCED'
 'activity_TOCFC' 'activity_TOCFCS' 'activity_TOCGR' 'activity_TOCHS'
 'activity_TOCLC' 'activity_TOCLF' 'activity_TOCLR' 'activity_TOCMN'
 'activity_TOCMS' 'activity_TOCMSS' 'activity_TOCNL' 'activity_TOCOG'
 'activity_TOCPA' 'activity_TOCPB' 'activity_TOCPBS' 'activity_TOCSE'
 'activity_TOCSR' 'activity_TOCWN']
Concatenated feature scores and names:
             Feature         Score
0    activity_TOCBM 

In [60]:
featureScores

,Feature,Score
0,activity_TOCBM,1.200000e+03
1,activity_TOCBR,1.084000e+03
2,activity_TOCBY,1.214000e+03
3,activity_TOCED,1.169000e+03
4,activity_TOCFC,1.099000e+03
5,activity_TOCFCS,1.218000e+03
6,activity_TOCGR,1.091000e+03
7,activity_TOCHS,1.198000e+03
8,activity_TOCLC,1.188000e+03
9,activity_TOCLF,1.210000e+03


In [61]:
print(featureScores.nlargest(10,'Score'))  #print 10 best features

            Feature         Score
22              NaN  5.489580e+06
25              NaN  1.148426e+04
18  activity_TOCPBS  1.219000e+03
13  activity_TOCMSS  1.218000e+03
5   activity_TOCFCS  1.218000e+03
12   activity_TOCMS  1.216000e+03
19   activity_TOCSE  1.216000e+03
2    activity_TOCBY  1.214000e+03
9    activity_TOCLF  1.210000e+03
0    activity_TOCBM  1.200000e+03


In [62]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)

ExtraTreesClassifier()

In [53]:
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers

[7.90193226e-03 9.56016159e-03 2.31899076e-03 1.15026602e-02
 1.19585832e-02 6.87337760e-04 7.61128843e-03 1.16711997e-02
 1.90191277e-03 2.21397061e-03 7.71110304e-03 1.53302002e-02
 1.20825299e-03 7.69472741e-04 1.51727555e-02 7.64049021e-03
 4.51129563e-02 9.78942897e-03 5.62166596e-04 1.15726405e-03
 4.34836318e-03 5.51244555e-03 5.90037260e-01 0.00000000e+00
 5.34047683e-02 1.74915035e-01 0.00000000e+00]


In [63]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

## Correlation Matrix with Heatmap

Correlation states how the features are related to each other or the target variable.

Correlation can be positive (increase in one value of feature increases the value of the target variable) or negative (increase in one value of feature decreases the value of the target variable)

Heatmap makes it easy to identify which features are most related to the target variable, we will plot heatmap of correlated features using the seaborn library.